In [1]:
from pathlib import Path
from typing import List

import fsspec
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pyproj import CRS, Transformer

from nsidc_projections.grid import to_cartopy
# from ros_database.ims_snow.ims_crs import IMS4kmGrid
from notebooks.utils import plot_ims_snow_and_ice, get_extent

DATAPATH = Path.home() / "src" / "rain_on_snow_database" / "data" / "test_data"

fs = fsspec.filesystem('file')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (ims_crs.py, line 228)

In [ ]:
test_point = {
    "Norton Sound": (-163.01706271914878, 64.54757445721512),
    "Cambridge Bay": (-105.05586791647777, 69.11608384685066),
    "Clyde River": (-68.59767446082255, 70.47531698648856),
    "Tiksi": (128.76156219329152, 71.70659836968615),
}

In [ ]:
filepath = list(DATAPATH.glob("*4km*.nc.gz"))[0]

ds_4km = xr.open_dataset(fs.open(filepath, compression="gzip"), decode_coords="all")
ds_4km

In [ ]:
6356752.314245

In [ ]:
filepath = list(DATAPATH.glob("*1km*.nc.gz"))[0]

ds_1km = xr.open_dataset(fs.open(filepath, compression="gzip"), decode_coords="all")
ds_1km

In [ ]:
plot_ims_snow_and_ice(ds_4km.IMS_Surface_Values.squeeze(), title="IMS Snow and Ice Cover 4km")

In [ ]:
plot_ims_snow_and_ice(ds_1km.IMS_Surface_Values.squeeze(), title="IMS Snow and Ice Cover 1km")

In [ ]:
location = "Norton Sound"

In [ ]:
extent = get_extent(test_point[location], ds_4km, fov=50000.)
ax = plot_ims_snow_and_ice(ds_4km.IMS_Surface_Values.squeeze(), title=f"IMS Snow and Ice 4km: {location}", extent=extent)
ax.scatter(*test_point[location], marker='o', color="red", transform=ccrs.PlateCarree())

In [ ]:
extent = get_extent(test_point[location], ds_1km, fov=50000.)
ax = plot_ims_snow_and_ice(ds_1km.IMS_Surface_Values.squeeze(), title=f"IMS Snow and Ice 1km: {location}", extent=extent)
ax.scatter(*test_point[location], marker='o', color="red", transform=ccrs.PlateCarree())

In [ ]:
xmin, xmax, ymin, ymax = ax.get_extent()
print(xmin, xmax, ymin, ymax)

In [ ]:
xg =  ds_1km.x.sel(x=slice(xmin,xmax)).values
yg =  ds_1km.y.sel(y=slice(ymin,ymax)).values

In [ ]:
proj = ccrs.Stereographic(central_latitude=90., 
                              central_longitude=-80., 
                              true_scale_latitude=60.)
coastline = cfeature.GSHHSFeature()

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection=proj)
ax.set_extent([xmin, xmax, ymin, ymax], proj)
ax.add_feature(coastline)

a, b = list(zip(*points))
#[ax.axvline(xxg) for xxg in xg];
#[ax.axhline(yyg) for yyg in yg];
ax.scatter(a, b, marker='.')

In [ ]:
from shapely.geometry import MultiPolygon, MultiPoint, Point

In [ ]:
multi_coast = MultiPolygon(coastline.geometries())

Test for Goluvin

In [ ]:
multi_coast.contains(Point(*test_point[location]))

In [ ]:
multi_coast.within(Point(*test_point[location]))

In [ ]:
multi_coast.contains_properly(Point(*test_point[location]))

In [ ]:
points = [IMS4kmGrid.xy_to_latlon(xxg, yyg) for xxg, yyg in product(xg, yg)]

In [ ]:
%%time
from itertools import product
is_land = np.array([multi_coast.contains_properly(Point(*point)) for point in points])

In [ ]:
proj = ccrs.Stereographic(central_latitude=90., 
                              central_longitude=-80., 
                              true_scale_latitude=60.)
coastline = cfeature.GSHHSFeature()

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(projection=proj)
ax.set_extent([xmin, xmax, ymin, ymax], proj)
ax.add_feature(coastline)

a, b = list(zip(*points))
#[ax.axvline(xxg) for xxg in xg];
#[ax.axhline(yyg) for yyg in yg];
ax.scatter(a, b, c=is_land, marker='.', transform=ccrs.PlateCarree())

In [ ]:
is_land